In [10]:
import obspy, os, glob

In [11]:
DATA_PATH = os.path.join(os.getcwd(), 'data')

In [14]:
stream_paths = glob.glob(os.path.join(DATA_PATH, '*/*.mseed'))

In [4]:
from obspy import Stream
from operator import add
from functools import reduce
from obspy import read

# Combine all streams into one
stream = reduce(add, [read(path) for path in stream_paths])

In [ ]:
print(stream)

In [5]:
import pandas as pd

noise_csv = pd.read_csv(os.path.join(DATA_PATH, 'Benz_catalog.csv'))
quake_csv = pd.read_csv(os.path.join(DATA_PATH, 'OK_2014-2015-2016.csv'))

In [6]:
def change_column_name(df, column_index, new_name):
    columns = df.columns.values
    columns[column_index] = new_name
    df.columns = columns
    
change_column_name(noise_csv, 0, 'EventNum')
noise_csv[:10]

,EventNum,Date,Time,Magnitude,Variance,origintime,utc_timestamp
0,0,02/15/2014,00:02:41,0.37,0.95,2014-02-15T00:02:41.000000Z,1.392423e+09
1,1,02/15/2014,00:03:45,-0.44,0.86,2014-02-15T00:03:45.000000Z,1.392423e+09
2,2,02/15/2014,00:08:07,-0.18,0.93,2014-02-15T00:08:07.000000Z,1.392423e+09
3,3,02/15/2014,00:12:52,0.10,0.93,2014-02-15T00:12:52.000000Z,1.392423e+09
4,4,02/15/2014,00:14:09,-0.47,0.89,2014-02-15T00:14:09.000000Z,1.392423e+09
5,5,02/15/2014,00:16:37,-0.76,0.62,2014-02-15T00:16:37.000000Z,1.392423e+09
6,6,02/15/2014,00:16:52,0.67,0.78,2014-02-15T00:16:52.000000Z,1.392423e+09
7,7,02/15/2014,00:19:12,3.51,0.98,2014-02-15T00:19:12.000000Z,1.392424e+09
8,8,02/15/2014,00:20:51,0.22,0.63,2014-02-15T00:20:51.000000Z,1.392424e+09
9,9,02/15/2014,00:21:06,-0.13,0.42,2014-02-15T00:21:06.000000Z,1.392424e+09


In [7]:
from obspy import UTCDateTime

def get_times(df):
    times = []
    for index, row in df.iterrows():
        month, day, year = row.Date.split('/')
        hour, minute, second = row.Time.split(':')
        args = map(int, [year, month, day, hour, minute, second])
        times.append(UTCDateTime(*args))      
    return times

In [17]:
times = get_times(noise_csv)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-83f837f77970>", line 1, in <module>
    times = get_times(noise_csv)
  File "<ipython-input-7-0965a25d6ca9>", line 5, in get_times
    for index, row in df.iterrows():
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py", line 698, in iterrows
    s = klass(v, index=columns, name=k)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/series.py", line 248, in __init__
    raise_cast_failure=True)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/series.py", line 2952, in _sanitize_array
    subarr = _try_cast(data, True)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/series.py", line 2919, in _try_cast
    subarr = maybe_cast_to_datetime(arr, dtype)
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/dtypes/cast.py", lin

KeyboardInterrupt: 

In [9]:
import datetime
from obspy import UTCDateTime

def get_time_range(stream):
    Min = UTCDateTime(year=datetime.MAXYEAR, month=1, day=1) 
    Max = UTCDateTime(year=datetime.MINYEAR, month=1, day=1)
    for trace in stream:
        Min = min(trace.stats.starttime,  Min)
        Max = max(trace.stats.starttime,  Max)
        
    return Min, Max

get_time_range(stream)

(2014-07-01T00:00:00.000000Z, 2014-07-29T20:14:53.205000Z)